### To run this code, please install PyIOmica, a python open multi - omics analysis platform. 
To install the current release from PyPI (Python Package Index) use pip:

pip install pyiomica

The github repository of PyIOmica: https://github.com/gmiaslab/pyiomica 

Instruction of PyIOmica, see: Sergii Domanskyi, Carlo Piermarocchi, George I Mias, PyIOmica: longitudinal omics analysis and trend identification, Bioinformatics, 2019, 1–2, doi: https://doi.org/10.1093/bioinformatics/btz896

Also need to install Louvain Community Detection package,

pip install python-louvain

The github repository of python-louvain: https://github.com/taynaud/python-louvain

In [6]:
import numpy as np
import networkx as nx
import community as cy
from networkx.algorithms import community
import itertools
from copy import deepcopy

from pyiomica import visualizationFunctions
from pyiomica import visibilityGraphCommunityDetection

from scipy import signal
import matplotlib.pyplot as plt

###no warnings
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import random

In [2]:
#function to plot the community as color bar
def __plotCommunityAsHeatmap(data, times, fileName,noRemovedData=None, title='', figsize=(8,4), cmap='jet',
                             graph_type='natural',weight=None, withsign=False, direction=None, cutoff=None):
    '''plot time series and community structure as heatmap, nodes in same community with same color
    
    Args:
        data: Numpy 2-D array of floats
        
        times: Numpy 1-D array of floats
        
        fileName: name of the figure file to save
        
        title: the figure title,default is empty
        
        noRemovedData: for uneven case, this data is the original data without remove any time points
                default is none
        
        figsize: tuple of int, Default (8,4)        
            Figure size in inches
        
        cmap: the color map to plot heatmap
        
        graph_type: string, default: 'natural'
            "horizontal", Horizontal Visibility Graph
            "natural",natural Visibility Graph
            "dual_horizontal", dual perspective horizontal visibility graph
            "dual_natural", dual perspective natural visibility graph
            
        weight: str, default:None
            None: no weighted
            'time': weight = abs(times[i] - times[j])
            'tan': weight = abs((data[i] - data[j])/(times[i] - times[j])) + 10**(-8)
            'distance': weight = A[i, j] = A[j, i] = ((data[i] - data[j])**2 + (times[i] - times[j])**2)**0.5
            
        withsign: boolean, Default False
            Whether to return the sign of adjacency matrix, 
            If True, the link from Natural perspective VG is positive, the link from reflected perspective VG is negative 
            Else, the are all positive
            
        direction:str, default is None, the direction that nodes aggregate to communities
            None: no specfic direction, e.g. both sieds
            left: nodes can only aggregate to the lefe side hubs, e.g. early hubs
            right: nodes can only aggregate to the right side hubs, e.g. later hubs
        
        cutoff: will be used to combine initial communities, e.g. whenever the shortest path length of 
            two adjacent hub nodes is smaller than cutoff, the communities with the two hub nodes will be combined.
            the cutoff can be int,float or string
            int or float: the percentile of all shortest path length distribution, between 0 ~ 100
            'auto': use optimized cutoff
            None: no cutoff
            the default is None
        
    Returns:
        None
        
    Usage:
        __plotCommunityAsHeatmap(data, times, 'Test.png', 'Test Data')
    '''
    methods = ['GN', 'LN','PL']
    G_nx, A = visibilityGraphCommunityDetection.createVisibilityGraph(data, times, graph_type=graph_type, weight=weight, withsign=withsign)
        
    community_pl = visibilityGraphCommunityDetection.communityDetectByPathLength(G_nx, direction=direction, cutoff = cutoff) 
    heatMapData = []
    
    if noRemovedData != None:
        lh = len(noRemovedData[0])
    else:
        lh = len(times)
    temp1 = np.zeros(lh)
    temp2 = np.zeros(lh)
    temp3 = np.zeros(lh) 


    comp = community.girvan_newman(G_nx)
    k = len(community_pl)    
    limited = itertools.takewhile(lambda c: len(c) <= k, comp)
    for communities in limited:
        community_gn = (list(sorted(c) for c in communities))
    for i, row in enumerate(community_gn):
        for j in row:
            temp3[int(G_nx.nodes[j]['timepoint'])] = i+1
            
    
    res3 = [element for element in temp3 if element != 0]    
    if temp3[0] == 0:
        temp3[0] = res3[0]        
    for i,e in enumerate(temp3):
        if e == 0:
            temp3[i] = temp3[i-1]    
    heatMapData.append(temp3)
    
    community_ln = cy.best_partition(G_nx)
    for key,value in community_ln.items():
        temp2[int(G_nx.nodes[key]['timepoint'])] = value + 1
    
    res2 = [element for element in temp2 if element != 0]    
    if temp2[0] == 0:
        temp2[0] = res2[0]        
    for i,e in enumerate(temp2):
        if e == 0:
            temp2[i] = temp2[i-1]  
    heatMapData.append(temp2)

    for i, row in enumerate(community_pl):
        for j in row:
            temp1[int(G_nx.nodes[j]['timepoint'])] = i + 1
    
    res1 = [element for element in temp1 if element != 0]    
    if temp1[0] == 0:
        temp1[0] = res1[0]        
    for i,e in enumerate(temp1):
        if e == 0:
            temp1[i] = temp1[i-1]           
    heatMapData.append(temp1)


    fig = plt.figure(figsize=figsize)
    ax1 = plt.subplot(211)
    ax1.bar(times, data, width=0.3,color='b')
    if noRemovedData != None:
        origT = noRemovedData[0]
        origD = noRemovedData[1]
        
        removT = [x for x in origT if x not in times]
        removD = origD[removT]
        
        ax1.bar(removT, removD, width=0.3,color='grey')
        
    if noRemovedData != None:
        length = len(noRemovedData[0])
        Ttimes = np.array(noRemovedData[0])
    else:
        length = len(data)
        Ttimes = np.array(times)
    
    ax1.axhline(y=0, color='k')
    ax1.set_ylabel('Signal Intensity', fontsize=16)
    ax1.set_xticks(np.arange(0,length,10))
    ax1.set_xticklabels(Ttimes[np.arange(0,length,10)],fontsize=16)
    ax1.set_yticks([])
    ax1.set_xlim(left=0,right=length)
    ax1.set_title(title,fontsize=20)
    

    ax2 = plt.subplot(212, sharex=ax1)
    #im2 = ax2.imshow(heatMapData)
    im2 = ax2.pcolor(heatMapData, cmap=cmap)
    ax2.set_xticks(np.arange(0,length,10))
    ax2.set_xticklabels(Ttimes[np.arange(0,length,10)],fontsize=16)
    ax2.set_yticks(np.arange(0,len(heatMapData),1), minor=True) 
    ax2.set_yticks(np.arange(0.5,len(heatMapData),1)) 
    ax2.set_yticklabels(methods,fontsize=16)
    ax2.grid(which="minor", color="w", axis='y', linestyle='-', linewidth=3)
    ax2.tick_params(which="minor", top=False, bottom=False, left=False,right=False)
    for edge, spine in ax2.spines.items():
        spine.set_visible(False)
        
    fig.tight_layout()
    fig.savefig(fileName, dpi=600)
    plt.close(fig)
    
    
    return None

#### Illustration of create weighted perspective visibility graph and community structure based on shortest path length community detection algorithm

In [7]:
### create time series
np.random.seed(11)
random.seed(11)
times = np.arange( 0, 2*np.pi, 0.35)
tp = list(range(len(times)))
data = 5*np.cos(times) + 2*np.random.random(len(times))

### plot time series
fig, ax = plt.subplots(figsize=(8,3))
ax.plot(tp,data)
ax.set_title('Time Series', fontdict={'color': 'k'},fontsize=20)
ax.set_xlabel('Times', fontsize=20)
ax.set_ylabel('Signal intensity', fontsize=20)
ax.set_xticks(tp)
ax.set_xticklabels([str(item) for item in np.round(tp,2)],fontsize=20, rotation=0)
ax.set_yticks([])

fig.tight_layout()
fig.savefig('./draft_fig/fig1/A.eps', dpi=600)
plt.close(fig)

### plot weighted Natural visibility graph, weight is  Euclidean distance
g_nx_NVG, A_NVG = visibilityGraphCommunityDetection.createVisibilityGraph(data,tp,"natural", weight = 'distance')
visualizationFunctions.PlotNVGBarGraph_Dual(A_NVG, data, tp,fileName='./draft_fig/fig1/B.eps',
                                            title = 'Natural Visibility Graph',fontsize=20,figsize=(8,3))

### plot reflected prespective weighted Natural visibility graph, weight is  Euclidean distance
g_nx_revNVG, A_revNVG = visibilityGraphCommunityDetection.createVisibilityGraph(-data,tp,"natural", weight = 'distance')
visualizationFunctions.PlotNVGBarGraph_Dual(A_revNVG, -data, tp,fileName='./draft_fig/fig1/C.eps',
                                            title='Reflected Prespective Natural Visibility Graph',fontsize=20,figsize=(8,3))

### plot dual prespective Natural visibility graph, weight is Euclidean distance
g_nx_dualNVG, A_dualNVG = visibilityGraphCommunityDetection.createVisibilityGraph(data,tp,"dual_natural", 
                                                                                  weight = 'distance', withsign=True)
visualizationFunctions.PlotNVGBarGraph_Dual(A_dualNVG, data, tp,fileName='./draft_fig/fig1/D.eps',
                                            title='Dual Prespective Natural Visibility Graph',fontsize=20,figsize=(10,4))

### plot line layout dual prespective Natural visibility graph with community structure, weight is Euclidean distance
communities = visibilityGraphCommunityDetection.communityDetectByPathLength(g_nx_dualNVG, direction = None, cutoff='auto')
com = (communities, g_nx_dualNVG)
visualizationFunctions.makeVisibilityGraph(data, tp, 'draft_fig/fig1', 'E', layout='line',communities=com, 
                       level=0.8,figsize = (10,6), extension='.eps')



graph type is: natural
weight is: distance
graph type is: natural
weight is: distance
graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 17]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
The PostScript backend does not support transparency; partially transparent artists will

#### plot community structure as heatmap, nodes in same community are with same color, comparing our algorithm with other traditional methods

In [16]:
t  = np.linspace( 0, 1, 150,endpoint=False)
tp = np.arange(len(t))

#Cosine signals

# 20 percent noise
data20a = np.cos(6*np.pi*t) + 0.2*(-1+2*np.random.random(len(t)))
__plotCommunityAsHeatmap(data20a, tp, './draft_fig/fig3/A.eps',title = '20 percent noise' ,cmap='jet',
                       graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')

#80 percent noise
data = np.cos(6*np.pi*t) + 0.8*(-1+2*np.random.random(len(t)))
__plotCommunityAsHeatmap(data, tp, './draft_fig/fig3/B.eps', title = '80 percent noise' ,cmap='jet',
                       graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')

# ### ramdonly remove 10 percent time points, 10 percent uneven samples
# np.random.shuffle(tp)
# tp_uneven = tp[:round(0.9*len(t))]
# tp_uneven = sorted(tp_uneven)
# data_uneven = data20a[tp_uneven]

# __plotCommunityAsHeatmap(data_uneven, tp_uneven, './draft_fig/fig3/C.eps',noRemovedData=(sorted(tp),data20a), title = '10 percent uneven samples' ,cmap='jet',
#                        graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')
#20%
np.random.shuffle(tp)
tp_uneven = tp[:round(0.8*len(t))]
tp_uneven = sorted(tp_uneven)
data_uneven = data20a[tp_uneven]

__plotCommunityAsHeatmap(data_uneven, tp_uneven, './draft_fig/fig3/C.eps',noRemovedData=(sorted(tp),data20a), title = '20 percent uneven samples' ,cmap='jet',
                       graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')


# ### randomly remove 40 percent time points, 40 percent uneven samples
# np.random.shuffle(tp)
# tp_uneven = tp[:round(0.6*len(t))]
# tp_uneven = sorted(tp_uneven)
# data_uneven = data20a[tp_uneven]

# __plotCommunityAsHeatmap(data_uneven, tp_uneven, './draft_fig/fig3/D.eps',noRemovedData=(sorted(tp),data20a), title = '40 percent uneven samples' ,cmap='jet',
#                        graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')
#80%
np.random.shuffle(tp)
tp_uneven = tp[:round(0.2*len(t))]
tp_uneven = sorted(tp_uneven)
data_uneven = data20a[tp_uneven]

__plotCommunityAsHeatmap(data_uneven, tp_uneven, './draft_fig/fig3/D.eps',noRemovedData=(sorted(tp),data20a), title = '80 percent uneven samples' ,cmap='jet',
                       graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')

graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 3, 48, 100, 149]
current cutoff is auto, the optimized percentiles cutoff is 1.351399 
graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 46, 102, 141, 146, 148, 149]
current cutoff is auto, the optimized percentiles cutoff is 2.714775 
graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 2, 78, 119]
current cutoff is auto, the optimized percentiles cutoff is 0.681629 
graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 17, 29]
current cutoff is auto, the optimized percentiles cutoff is 32.651990 


In [12]:
###square wave signal
t  = np.linspace( 0, 1, 150, endpoint=False)
tp = np.arange(len(t))

#20 percent noise
data20b = signal.square(6*np.pi*t) + 0.2 * (-1+2*np.random.random(len(t)))
__plotCommunityAsHeatmap(data20b, tp, './draft_fig/fig3/E.eps', title = '20 percent noise' ,cmap='jet',
                       graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')

#80 percent noise
data = signal.square(6*np.pi*t) + 0.8 * (-1+2*np.random.random(len(t))) 
__plotCommunityAsHeatmap(data, tp, './draft_fig/fig3/F.eps', title = '80 percent noise' ,cmap='jet',
                       graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')

# #10 percent uneven samples
# np.random.shuffle(tp)
# tp_uneven = tp[:round(0.9*len(t))]  
# tp_uneven = sorted(tp_uneven)
# data_uneven = data20b[tp_uneven]
# __plotCommunityAsHeatmap(data_uneven, tp_uneven, './draft_fig/fig3/G.eps',noRemovedData=(sorted(tp),data20b), title = '10 percent uneven samples' ,cmap='jet',
#                        graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')

#20 percent uneven samples
np.random.shuffle(tp)
tp_uneven = tp[:round(0.8*len(t))]  
tp_uneven = sorted(tp_uneven)
data_uneven = data20b[tp_uneven]
__plotCommunityAsHeatmap(data_uneven, tp_uneven, './draft_fig/fig3/G.eps',noRemovedData=(sorted(tp),data20b), title = '20 percent uneven samples' ,cmap='jet',
                       graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')

# #40 percent uneven samples
# np.random.shuffle(tp)
# tp_uneven = tp[:round(0.6*len(t))] 
# tp_uneven = sorted(tp_uneven)
# data_uneven = data20b[tp_uneven]
# __plotCommunityAsHeatmap(data_uneven, tp_uneven, './draft_fig/fig3/H.eps',noRemovedData=(sorted(tp),data20b), title = '40 percent uneven samples' ,cmap='jet',
#                        graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')

#80 percent uneven samples
np.random.shuffle(tp)
tp_uneven = tp[:round(0.2*len(t))] 
tp_uneven = sorted(tp_uneven)
data_uneven = data20b[tp_uneven]
__plotCommunityAsHeatmap(data_uneven, tp_uneven, './draft_fig/fig3/H.eps',noRemovedData=(sorted(tp),data20b), title = '80 percent uneven samples' ,cmap='jet',
                       graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')

graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 25, 30, 40, 42, 45, 49, 75, 138, 141, 148, 149]
current cutoff is auto, the optimized percentiles cutoff is 16.884744 
graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 25, 28, 44, 75, 77, 81, 90, 99, 125, 144, 147, 149]
current cutoff is auto, the optimized percentiles cutoff is 20.261213 
graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 20, 36, 39, 60, 108, 111, 118, 119]
current cutoff is auto, the optimized percentiles cutoff is 18.236870 
graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 4, 28, 29]
current cutoff is auto, the optimized percentiles cutoff is 5.185981 


In [13]:
###sawtooth wave signal
t  = np.linspace( 0, 1, 150, endpoint=False)
tp = np.arange(len(t))

#20 percent noise
data20c = signal.sawtooth(6*np.pi*t, 0)+ 0.2 * (-1+2*np.random.random(len(t))) #20 percent noise
__plotCommunityAsHeatmap(data20c, tp, './draft_fig/fig3/I.eps', title = '20 percent noise' ,cmap='jet',
                       graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')

#80 percent noise
data = signal.sawtooth(6*np.pi*t,0)+ 0.8 * (-1+2*np.random.random(len(t)))
__plotCommunityAsHeatmap(data, tp, './draft_fig/fig3/J.eps', title = '80 percent noise' ,cmap='jet',
                       graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')

#  #10 percent uneven samples
# np.random.shuffle(tp)
# tp_uneven = tp[:round(0.9*len(t))]  #10 percent uneven samples
# tp_uneven = sorted(tp_uneven)
# data_uneven = data20c[tp_uneven]
# __plotCommunityAsHeatmap(data_uneven, tp_uneven, './draft_fig/fig3/K.eps',noRemovedData=(sorted(tp),data20c),title = '10 percent uneven samples' ,cmap='jet',
#                        graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')

 #20 percent uneven samples
np.random.shuffle(tp)
tp_uneven = tp[:round(0.8*len(t))]  #20 percent uneven samples
tp_uneven = sorted(tp_uneven)
data_uneven = data20c[tp_uneven]
__plotCommunityAsHeatmap(data_uneven, tp_uneven, './draft_fig/fig3/K.eps',noRemovedData=(sorted(tp),data20c),title = '20 percent uneven samples' ,cmap='jet',
                       graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')

# #40 percent uneven samples
# np.random.shuffle(tp)
# tp_uneven = tp[:round(0.6*len(t))] 
# tp_uneven = sorted(tp_uneven)
# data_uneven = data20c[tp_uneven]
# __plotCommunityAsHeatmap(data_uneven, tp_uneven, './draft_fig/fig3/L.eps',noRemovedData=(sorted(tp),data20c),title = '40 percent uneven samples' ,cmap='jet',
#                        graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')

 #80 percent uneven samples
np.random.shuffle(tp)
tp_uneven = tp[:round(0.2*len(t))]  #80 percent uneven samples
tp_uneven = sorted(tp_uneven)
data_uneven = data20c[tp_uneven]
__plotCommunityAsHeatmap(data_uneven, tp_uneven, './draft_fig/fig3/L.eps', noRemovedData=(sorted(tp),data20c), title = '80 percent uneven samples' ,cmap='jet',
                       graph_type='dual_natural', weight='distance', direction=None, cutoff='auto')


graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 100, 123, 134, 141, 149]
current cutoff is auto, the optimized percentiles cutoff is 14.872419 
graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 2, 6, 9, 10, 51, 103, 140, 149]
current cutoff is auto, the optimized percentiles cutoff is 5.406599 
graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 4, 6, 8, 19, 36, 79, 113, 117, 119]
current cutoff is auto, the optimized percentiles cutoff is 15.549956 
graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 8, 22, 23, 26, 29]
current cutoff is auto, the optimized percentiles cutoff is 11.199190 


In [14]:
#no noise
t  = np.linspace( 0, 1, 150, endpoint=False)
tp = np.arange(len(t))

data0a = np.cos(6*np.pi*t) 
__plotCommunityAsHeatmap(data0a, tp, './draft_fig/fig2/A.eps',title = 'Cosine Signal' ,cmap='jet',
                       graph_type='dual_natural', weight='distance', direction='left', cutoff='auto')


data0b = signal.square(6*np.pi*t) 
__plotCommunityAsHeatmap(data0b, tp, './draft_fig/fig2/B.eps', title = 'Square Wave Signal' ,cmap='jet',
                       graph_type='dual_natural', weight='distance', direction='left', cutoff='auto')

# data0a = np.sin(6*np.pi*t) 
# __plotCommunityAsHeatmap(data0a, tp, './draft_fig/fig2/Aex.eps',title = 'Sin Signal' ,cmap='jet',
#                        graph_type='dual_natural', weight='distance', direction='left', cutoff='auto')

graph type is: dual_natural
weight is: distance
direction type is: left
the shortest path is: [0, 50, 100, 149]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 
graph type is: dual_natural
weight is: distance
direction type is: left
the shortest path is: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 149]
current cutoff is auto, the optimized percentiles cutoff is 16.882781 


In [12]:
def __compareCommunityOfDiffWeights(data, times, fileName,noRemovedData=None, title='', figsize=(8,4),
                                    cmap='jet',graph_type='natural', direction=None, cutoff=None, 
                                    ySignalTick=False, xtickInterval = 20, f=1):
    '''plot time series and community structure as heatmap, nodes in same community with same color
       compare different weights
    
    Args:
        data: Numpy 2-D array of floats
        
        times: Numpy 1-D array of floats
        
        fileName: name of the figure file to save
        
        title: the figure title,default is empty
        
        noRemovedData: for uneven case, this data is the original data without remove any time points
                default is none
        
        figsize: tuple of int, Default (8,4)        
            Figure size in inches
        
        cmap: the color map to plot heatmap
        
        graph_type: string, default: 'natural'
            "horizontal", Horizontal Visibility Graph
            "natural",natural Visibility Graph
            "dual_horizontal", dual perspective horizontal visibility graph
            "dual_natural", dual perspective natural visibility graph
            
 
        direction:str, default is None, the direction that nodes aggregate to communities
            None: no specfic direction, e.g. both sieds
            left: nodes can only aggregate to the lefe side hubs, e.g. early hubs
            right: nodes can only aggregate to the right side hubs, e.g. later hubs
        
        cutoff: will be used to combine initial communities, e.g. whenever the shortest path length of 
            two adjacent hub nodes is smaller than cutoff, the communities with the two hub nodes will be combined.
            the cutoff can be int,float or string
            int or float: the percentile of all shortest path length distribution, between 0 ~ 100
            'auto': use optimized cutoff
            None: no cutoff
            the default is None
        
    Returns:
        None
        
    Usage:
        __compareCommunityOfDiffWeights(data, times, 'Test.png', 'Test Data')
    '''
    methods = ['None', u'Δ Time','Tangent','Euclidean Distance']
    G_n, A = visibilityGraphCommunityDetection.createVisibilityGraph(data, times, graph_type=graph_type, weight=None)        
    c_n = visibilityGraphCommunityDetection.communityDetectByPathLength(G_n, direction=direction, cutoff = cutoff)
    G_tm, A = visibilityGraphCommunityDetection.createVisibilityGraph(data, times, graph_type=graph_type, weight='time') 
    c_tm = visibilityGraphCommunityDetection.communityDetectByPathLength(G_tm, direction=direction, cutoff = cutoff)
    G_tn, A = visibilityGraphCommunityDetection.createVisibilityGraph(data, times, graph_type=graph_type, weight='tan') 
    c_tn = visibilityGraphCommunityDetection.communityDetectByPathLength(G_tn, direction=direction, cutoff = cutoff)
    G_dis, A = visibilityGraphCommunityDetection.createVisibilityGraph(data, times, graph_type=graph_type, weight='distance') 
    c_dis = visibilityGraphCommunityDetection.communityDetectByPathLength(G_dis, direction=direction, cutoff = cutoff)

    
    heatMapData = []
    
    if noRemovedData != None:
        lh = len(noRemovedData[0])
    else:
        lh = len(times)
    
    def __get_heatmap_array(G, community,lh):
        
        temp = np.zeros(lh)
        for i, row in enumerate(community):
            for j in row:
                temp[int(f*float(G.nodes[j]['timepoint']))] = i + 1
        
        res = [element for element in temp if element != 0]    
        if temp[0] == 0:
            temp[0] = res[0]        
        for i,e in enumerate(temp):
            if e == 0:
                temp[i] = temp[i-1] 

        return temp
    
    heatMapData.append(__get_heatmap_array(G_n,c_n,lh))
    heatMapData.append(__get_heatmap_array(G_tm,c_tm,lh))  
    heatMapData.append(__get_heatmap_array(G_tn,c_tn,lh))    
    heatMapData.append(__get_heatmap_array(G_dis,c_dis,lh))

    fig = plt.figure(figsize=figsize)
    ax1 = plt.subplot(211)
    T = [int(round(x*f)) for x in times ]
    ax1.bar(T, data, width=0.3,color='b')
    if noRemovedData != None:
        origT = noRemovedData[0]
        origD = noRemovedData[1]
        
        removT = [x for x in origT if x not in times]
        removD = origD[removT]
        rT = [int(round(x*f)) for x in removT ]
        
        ax1.bar(rT, removD, width=0.3,color='grey')
        
    if noRemovedData != None:
        length = len(noRemovedData[0])
        Ttimes = np.arange(length)
    else:
        length = len(data)
        Ttimes = np.arange(len(times))
    
    ax1.axhline(y=0, color='k')
    ax1.set_ylabel('Signal Intensity', fontsize=16)
    ax1.set_xticks(np.arange(0,length,xtickInterval))
    ax1.set_xticklabels(Ttimes[np.arange(0,length,xtickInterval)],fontsize=16)
    if ySignalTick == False:
        ax1.set_yticks([])
    ax1.set_xlim(left=0,right=length)
    ax1.set_title(title,fontsize=20)
    

    ax2 = plt.subplot(212, sharex=ax1)
    #im2 = ax2.imshow(heatMapData)
    im2 = ax2.pcolor(heatMapData, cmap=cmap)
    ax2.set_xticks(np.arange(0,length,xtickInterval))
    ax2.set_xticklabels(Ttimes[np.arange(0,length,xtickInterval)],fontsize=16)
    ax2.set_yticks(np.arange(0,len(heatMapData),1), minor=True) 
    ax2.set_yticks(np.arange(0.5,len(heatMapData),1)) 
    ax2.set_yticklabels(methods,fontsize=16)
    ax2.grid(which="minor", color="w", axis='y', linestyle='-', linewidth=3)
    ax2.tick_params(which="minor", top=False, bottom=False, left=False,right=False)
    for edge, spine in ax2.spines.items():
        spine.set_visible(False)
        
    fig.tight_layout()
    fig.savefig(fileName, dpi=600)
    plt.close(fig)
        
    return None

In [14]:
# compare different weights
# NB: remove seed to run new simulations
np.random.seed(20) 
random.seed(20)

t  = np.linspace( 0, 1, 150,endpoint=False)
tp = np.arange(len(t))

cf = 'auto'
graphtype = 'dual_natural'


#Cosine signals
data00 = np.cos(6*np.pi*t) 
__compareCommunityOfDiffWeights(data00, tp, './draft_fig/fig4/A1.eps',title = '0 percent noise' ,cmap='jet',
                       graph_type=graphtype,direction=None, cutoff=cf)



# 20 percent noise
data20a = np.cos(6*np.pi*t) + 0.2*(-1+2*np.random.random(len(t)))
__compareCommunityOfDiffWeights(data20a, tp, './draft_fig/fig4/A2.eps',title = '20 percent noise' ,cmap='jet',
                       graph_type=graphtype,direction=None, cutoff=cf)

#80 percent noise
data = np.cos(6*np.pi*t) + 0.8*(-1+2*np.random.random(len(t)))
__compareCommunityOfDiffWeights(data, tp, './draft_fig/fig4/A3.eps', title = '80 percent noise' ,cmap='jet',
                       graph_type=graphtype, direction=None, cutoff=cf)


#20 missing%
np.random.shuffle(tp)
tp_uneven = tp[:round(0.8*len(t))]
tp_uneven = sorted(tp_uneven)
data_uneven = data20a[tp_uneven]

__compareCommunityOfDiffWeights(data_uneven, tp_uneven, './draft_fig/fig4/A4.eps',noRemovedData=(sorted(tp),data20a), title = '20 percent missing data' ,cmap='jet',
                       graph_type=graphtype, direction=None, cutoff=cf)

#80 missing%
np.random.shuffle(tp)
tp_uneven = tp[:round(0.2*len(t))]
tp_uneven = sorted(tp_uneven)
data_uneven = data20a[tp_uneven]

__compareCommunityOfDiffWeights(data_uneven, tp_uneven, './draft_fig/fig4/A5.eps',noRemovedData=(sorted(tp),data20a), title = '80 percent missing data' ,cmap='jet',
                       graph_type=graphtype, direction=None, cutoff=cf)




graph type is: dual_natural
weight is: None
direction type is: None
the shortest path is: [0, 50, 100, 149]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 
graph type is: dual_natural
weight is: time
direction type is: None
the shortest path is: [0, 50, 100, 149]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 
graph type is: dual_natural
weight is: tan
direction type is: None
the shortest path is: [0, 50, 100, 149]
current cutoff is auto, the optimized percentiles cutoff is 0.287685 
graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 50, 100, 149]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 
graph type is: dual_natural
weight is: None
direction type is: None
the shortest path is: [0, 1, 100, 149]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 
graph type is: dual_natural
weight is: time
direction type is: None
the shortest path is: [0, 1, 100, 149]
c

In [15]:
# different intensity and frequency
# NB: remove seed to run new simulations
np.random.seed(20)
random.seed(20)

t  = np.linspace(0, 1, 150,endpoint=False)
tp = np.arange(len(t))

cf = 'auto'
graphtype = 'dual_natural'
factor = 10
yTick = True
xinter = 20

data00 = np.cos(6*np.pi*t) 
__compareCommunityOfDiffWeights(data00, tp, './draft_fig/fig4/B1.eps',title = 'amplitude=1,frequency=1 ' ,cmap='jet',
                       graph_type=graphtype,direction=None, cutoff=cf,ySignalTick=yTick)


data_a50 = factor*np.cos(6*np.pi*t)
__compareCommunityOfDiffWeights(data_a50, tp, './draft_fig/fig4/B2.eps',title ='amplitude='+str(factor)+',frequency=1' ,cmap='jet',
                       graph_type=graphtype,direction=None, cutoff=cf,ySignalTick=yTick)


factor = 100
t  = np.linspace(0, 1, 150,endpoint=False)

data_a100 = factor*np.cos(6*np.pi*t)
__compareCommunityOfDiffWeights(data_a100, tp, './draft_fig/fig4/B3.eps',title ='amplitude='+str(factor)+',frequency=1' ,cmap='jet',
                       graph_type=graphtype,direction=None, cutoff=cf,ySignalTick=yTick)

f = 5

data_f5 = np.cos(f*6*np.pi*t)
__compareCommunityOfDiffWeights(data_f5, tp, './draft_fig/fig4/B4.eps',title = 'amplitude=1,frequency='+str(f) ,cmap='jet',
                       graph_type=graphtype,direction=None, cutoff=cf,ySignalTick=yTick,xtickInterval=xinter)

f = 10
data_f10= np.cos(f*6*np.pi*t) 
__compareCommunityOfDiffWeights(data_f10, tp, './draft_fig/fig4/B5.eps',title = 'amplitude=1,frequency='+str(f) ,cmap='jet',
                       graph_type=graphtype,direction=None, cutoff=cf,ySignalTick=yTick,xtickInterval=xinter)

graph type is: dual_natural
weight is: None
direction type is: None
the shortest path is: [0, 50, 100, 149]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 
graph type is: dual_natural
weight is: time
direction type is: None
the shortest path is: [0, 50, 100, 149]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 
graph type is: dual_natural
weight is: tan
direction type is: None
the shortest path is: [0, 50, 100, 149]
current cutoff is auto, the optimized percentiles cutoff is 0.287685 
graph type is: dual_natural
weight is: distance
direction type is: None
the shortest path is: [0, 50, 100, 149]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 
graph type is: dual_natural
weight is: None
direction type is: None
the shortest path is: [0, 50, 100, 149]
current cutoff is auto, the optimized percentiles cutoff is 0.000000 
graph type is: dual_natural
weight is: time
direction type is: None
the shortest path is: [0, 50, 100, 149]